In [6]:
import pandas as pd
import sqlite3
import os

In [7]:
#Checking column names and looking for join potential
df_NFL_Draft_2025 = pd.read_csv('NFL_Draft_2025.csv')
print(df_NFL_Draft_2025.columns.tolist())


['CollegeAthleteId', 'NflAthleteId', 'CollegeId', 'CollegeTeam', 'CollegeConference', 'NflTeamId', 'NflTeam', 'Year', 'Overall', 'Round', 'Pick', 'Name', 'Position', 'Height', 'Weight', 'PreDraftRanking', 'PreDraftPositionRanking', 'PreDraftGrade', 'HometownInfo City', 'HometownInfo State', 'HometownInfo Country', 'HometownInfo Latitude', 'HometownInfo Longitude', 'HometownInfo CountyFips']


In [8]:
# Checking column names and looking for join potential
df_Recruiting_Ranking_2022 = pd.read_csv('Recruiting_Ranking_2022.csv')
print(df_Recruiting_Ranking_2022.columns)

Index(['Id', 'AthleteId', 'RecruitType', 'Year', 'Ranking', 'Name', 'School',
       'CommittedTo', 'Position', 'Height', 'Weight', 'Stars', 'Rating',
       'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')


In [3]:
#Combining Highschool recruiting ranking csv for years 202019-2023 ( there should be 4 people per rank)

files_to_combine = [
    'Recruiting_Ranking_2020.csv',
    'Recruiting_Ranking_2021.csv',
    'Recruiting_Ranking_2022.csv',
    'Recruiting_Ranking_2019.csv'
]

Recruiting_Ranking = []

for file in files_to_combine:
    College = pd.read_csv(file)
    Recruiting_Ranking.append(College)

Recruiting_Rank_Combined = pd.concat(Recruiting_Ranking, ignore_index=True)

print(Recruiting_Rank_Combined.head(10))

      Id  AthleteId RecruitType  Year  Ranking               Name  \
0  61569  4428988.0  HighSchool  2020      1.0       Bryan Bresee   
1  61570  4685720.0  HighSchool  2020      2.0        Bryce Young   
2  61571  4430800.0  HighSchool  2020      3.0     Julian Fleming   
3  61572  4428992.0  HighSchool  2020      4.0        Kelee Ringo   
4  61573  4428994.0  HighSchool  2020      5.0       Arik Gilbert   
5  61574  4428986.0  HighSchool  2020      6.0       Justin Flowe   
6  61576  4428996.0  HighSchool  2020      8.0       Jordan Burch   
7  61577  4428991.0  HighSchool  2020      9.0  Paris Johnson Jr.   
8  61578        NaN  HighSchool  2020     10.0      DJ Uiagalelei   
9  61579  4428990.0  HighSchool  2020     11.0    Broderick Jones   

              School     CommittedTo Position  Height  Weight  Stars  Rating  \
0           Damascus         Clemson       DT    77.0   290.0      5  0.9995   
1          Mater Dei         Alabama     DUAL    71.0   183.0      5  0.9994   


In [5]:
# Combining NFL Draft results from 2022-2025

files_to_combine_draft = [
    'NFL_Draft_2022.csv',
    'NFL_Draft_2023.csv',
    'NFL_Draft_2024.csv',
    'NFL_Draft_2025.csv'
]
NFL_Draft = []
for file in files_to_combine_draft:
    NFL = pd.read_csv(file)
    NFL_Draft.append(NFL)

NFL_Draft_Combined = pd.concat(NFL_Draft, ignore_index=True)

print(NFL_Draft_Combined.head(10))

   CollegeAthleteId  NflAthleteId  CollegeId        CollegeTeam  \
0         4426349.0        105673         61            Georgia   
1         4372099.0        105537        130           Michigan   
2         4426434.0        105534         99                LSU   
3         4427250.0        105558       2132         Cincinnati   
4         4426326.0        105533       2483             Oregon   
5         4427132.0        105589        152           NC State   
6         4567134.0        105535        333            Alabama   
7         4426502.0        105554         30                USC   
8         4426376.0        105539        344  Mississippi State   
9         4569618.0        105540        194         Ohio State   

   CollegeConference  NflTeamId       NflTeam  Year  Overall  Round  ...  \
0                SEC         30  Jacksonville  2022        1      1  ...   
1            Big Ten          8       Detroit  2022        2      1  ...   
2                SEC         34   

In [ ]:
# Looking at results
print(NFL_Draft_Combined.columns)

Index(['CollegeAthleteId', 'NflAthleteId', 'CollegeId', 'CollegeTeam',
       'CollegeConference', 'NflTeamId', 'NflTeam', 'Year', 'Overall', 'Round',
       'Pick', 'Name', 'Position', 'Height', 'Weight', 'PreDraftRanking',
       'PreDraftPositionRanking', 'PreDraftGrade', 'HometownInfo City',
       'HometownInfo State', 'HometownInfo Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo CountyFips'],
      dtype='object')


In [8]:
# looking at results
print(NFL_Draft_Combined.head(15))

    CollegeAthleteId  NflAthleteId  CollegeId        CollegeTeam  \
0          4426349.0        105673         61            Georgia   
1          4372099.0        105537        130           Michigan   
2          4426434.0        105534         99                LSU   
3          4427250.0        105558       2132         Cincinnati   
4          4426326.0        105533       2483             Oregon   
5          4427132.0        105589        152           NC State   
6          4567134.0        105535        333            Alabama   
7          4426502.0        105554         30                USC   
8          4426376.0        105539        344  Mississippi State   
9          4569618.0        105540        194         Ohio State   
10         4361370.0        105544        194         Ohio State   
11         4426388.0        105553        333            Alabama   
12         4381558.0        105548         61            Georgia   
13         4575517.0        105536         87   

In [ ]:
#Looking at shape to see if requirements were met
NFL_Draft_Combined.shape

(1035, 24)

In [12]:
Recruiting_Rank_Combined.columns

Index(['Id', 'AthleteId', 'RecruitType', 'Year', 'Ranking', 'Name', 'School',
       'CommittedTo', 'Position', 'Height', 'Weight', 'Stars', 'Rating',
       'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')

In [ ]:
# Renaming columns for consistency
Recruiting_Rank_Combined.rename(columns={'AthleteId': 'CollegeAthleteId'}, inplace=True)

In [14]:
Recruiting_Rank_Combined.columns

Index(['Id', 'CollegeAthleteId', 'RecruitType', 'Year', 'Ranking', 'Name',
       'School', 'CommittedTo', 'Position', 'Height', 'Weight', 'Stars',
       'Rating', 'City', 'StateProvince', 'Country', 'HometownInfo Latitude',
       'HometownInfo Longitude', 'HometownInfo FipsCode'],
      dtype='object')

In [18]:
Recruiting_Rank_Combined.drop(['HometownInfo FipsCode', 'RecruitType'], axis=1, inplace=True)

